TODO: visualize the eigenvectors of the mass matrix as "vibrational modes"; why are the eigenvalue so much larger for the full mass matrix than for the lumped mass matrix?

In [ ]:
import sys; sys.path.append('..')
import elastic_rods, sparse_matrices, pickle
import numpy as np
from numpy.linalg import norm
from scipy.sparse import csc_matrix, diags
from scipy.sparse.linalg import eigsh
import time

r = pickle.load(open('../data/bad_rod.pkl', 'rb'))
r.updateSourceFrame()
r.setMaterial(elastic_rods.RodMaterial('ellipse', 2000, 0.3, [0.01, 0.005]))
fixedVars = [0, 1, 2, 77, 150, 151, 152]

elastic_rods.compute_equilibrium(r, verbose=True, fixedVars=fixedVars, niter=40)

In [ ]:
r.updateSourceFrame()
M = r.massMatrix()
Mlumped = r.lumpedMassMatrix()

x = (2 * np.random.random_sample(size=r.numDoF()) - 1.0)
x = x / norm(x)

t = time.time()
power_iterations = []
for i in range(1000):
    x = M.apply(x)
    lam = norm(x)
    x = x / lam
    power_iterations.append(lam)
elapsed = time.time() - t
print("Power method took {}s".format(elapsed))

power_iterations = np.array(power_iterations)
print(power_iterations[-1])

M.reflectUpperTriangle()
M_csc = csc_matrix(M.compressedColumn())
t = time.time()
lambdas, evecs = eigsh(M_csc, 1, which='LM')
elapsed = time.time() - t
print("eigsh took {}s".format(elapsed))

max(lambdas)

In [ ]:
Mlumped[-1]

In [ ]:
import matplotlib.pyplot as plt
plt.loglog(np.abs(power_iterations - max(lambdas)))
plt.show()

In [ ]:
plt.spy(M_csc)

In [ ]:
np.max(sum(M_csc))

In [ ]:
from mode_viewer import ModeViewer
mview = ModeViewer(r, evecs, lambdas)

In [ ]:
mview.show()

In [ ]:
Mlumped

In [ ]:
mat = r.material(0)

In [ ]:
mat.twistingStiffness

In [ ]:
mat = elastic_rods.RodMaterial()
mat.setEllipse(200, 0.3, 0.01, 0.005)

In [ ]:
from bending_validation import *
L, a, n = 1, 0.1, 51
perturb = False
r, fixedVars = bendingTestRod(L, a, n)
pts, thetas = initialConfiguration(L, a, n, perturb)
thetas = np.ones(n - 1) * (np.pi / 2)
r.setDeformedConfiguration(pts, thetas)

In [ ]:
r.lumpedMassMatrix()